# HW3: Netflix Data Analysis

In this homework assignment, you will analyze the netflix prize data. The data consist of 100,480,50 movie ratings on a scale from 0 to 5 stars. The reveiws are distributed across 17,770 movies and 480,189. We have provided the training data as a sparse matrix where the row corresponds to the movie ID and the column corresponds to the user ID. A seperate file contains the title and year of release for each movie. The original, raw data consists of multiple lists of tuples; each list is a seperate movie and each tuple is User ID, Rating, and Rating Year. 
The original data can be downloaded here: https://archive.org/download/nf_prize_dataset.tar
Further information about the netflix prize is available online: 
https://en.wikipedia.org/wiki/Netflix_Prize
https://www.netflixprize.com/

In [278]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, csc_matrix
from scipy import spatial
import scipy.sparse as sparse
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [368]:
# This file consists of titles and release years associated with each ID
movie_titles = pd.read_csv('movie_titles.txt', header = None, names = ['ID','Year','Name'])
print(movie_titles.head())
print(movie_titles.shape)

movie_by_id = {}
for id, name, year in zip(movie_titles['ID'], movie_titles['Name'], movie_titles['Year']):
    if not (np.isnan(year)):
        year = str(int(year))
    else:
        year = 'NaN'
    movie_by_id[id] = name + ' ' + '(' + year + ')'

   ID    Year                          Name
0   1  2003.0               Dinosaur Planet
1   2  2004.0    Isle of Man TT 2004 Review
2   3  1997.0                     Character
3   4  1994.0  Paula Abdul's Get Up & Dance
4   5  2004.0      The Rise and Fall of ECW
(17770, 3)


In [369]:
movie_genres = pd.read_csv('../onehot_all_movie_genres.csv', header = 0)
print(movie_genres.shape)

(17770, 28)


In [4]:
# This file is a sparse matrix of movies by user, with each element a rating (1-5) or nonresponse (0)
ratings_csr = scipy.sparse.load_npz('netflix_full_csr.npz')
print(ratings_csr.shape)

(17771, 2649430)


To avoid memory overflow errors we have randomly subsampled the data. Some computers can handle the full dataset (e.g. 2017 Macbook Pro can perform SVD on the full dataset). Older computers likely need to subsample the data. You can consider using Princeton computing resources and clusters to perform more computationally expensive analysis.

In [370]:
# Filter the matris to remove rows with NO REVIEWS
start = time.time()
ratings_csc = ratings_csr.T
print 'before removing users with no reviews: ', ratings_csc.shape
non_zero_users_csc = ratings_csc[(ratings_csc.getnnz(axis=1) != 0)]
print non_zero_users_csc.shape

finish = time.time()
print 'finished in %.2f seconds' % (finish - start)

before removing users with no reviews:  (2649430, 17771)
(480189, 17771)
finished in 29.81 seconds


A common methods for analyzing large datasets is dimension reduction. Here we perform a truncated SVD suited for sparse datasets and analyze which movies are associated with different latent dimensions

In [7]:
from sklearn.decomposition import TruncatedSVD

In [7]:
n_components = 5
svd = TruncatedSVD(n_components = n_components)

In [8]:
Z = svd.fit_transform(ratings_small)

In [9]:
components = svd.components_

In [10]:
print(svd.explained_variance_ratio_)

[0.22315634 0.02998073 0.01984643 0.01672574 0.01252159]


In [ ]:
for i in range(0,n_components):
    Z_sort = np.argsort(np.abs(Z[:,i]))
    print('Component ' + str(i))
    for j in range(1,10):
        movie_index = Z_sort[-j]
        movie_title = movie_titles[movie_titles['ID'] == movie_index]['Name']
        movie_weight = Z[movie_index,i]
        print(str(movie_title) + ': ' + str(movie_weight))
    print(' ')

In [382]:
# construct a dictionary to store number of reviews per user
print non_zero_users_csc.shape
non_zero_users_csr = csr_matrix(non_zero_users_csc)

reviews_by_user = {}
for u in range(non_zero_users_csr.shape[0]):
    reviews_by_user[u] = non_zero_users_csr[u].nnz

(480189, 17771)


In [380]:
s = sorted(reviews_by_user.keys(), key=lambda x: reviews_by_user[x], reverse=True)[:10]
print 'highest amount of reviews per user:', [reviews_by_user[i] for i in s]
print [i for i in s]

highest amount of reviews per user: [17653, 17436, 16565, 15813, 14831, 9821, 9768, 9739, 9064, 8881]
[55373, 70466, 442139, 301823, 383961, 265129, 297513, 238656, 472465, 350357]


In [409]:
ratings_small.shape

(17771, 480189)

In [416]:
# count the number of reviews for each film and store in review_nums list
review_nums = []
for i in range(ratings_small.shape[0]):
    num_reviews = ratings_small[i].nnz
    review_nums.append((i, num_reviews, np.sum(ratings_small[i]) / num_reviews))

# Print the top movies by number of reviews 
s = sorted(review_nums, key=lambda x: x[2])
print '#revs\tavg.\tmovie'
for movie_id, num, avg_review in s[-20:]:
    print '%s\t%0.4f\t%s' % (num, avg_review, movie_by_id[movie_id])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in divide


#revs	avg.	movie
681	4.5389	Fruits Basket (2001)
17292	4.5426	The Simpsons: Season 5 (1993)
92470	4.5437	Star Wars: Episode V: The Empire Strikes Back (1980)
134284	4.5451	Lord of the Rings: The Return of the King (2003)
125	4.5520	Lord of the Rings: The Return of the King: Extended Edition: Bonus Material (2003)
1883	4.5544	Inu-Yasha (2000)
8426	4.5813	The Simpsons: Season 6 (1994)
6621	4.5824	Arrested Development: Season 2 (2004)
220	4.5864	Ghost in the Shell: Stand Alone Complex: 2nd Gig (2005)
1238	4.5921	Veronica Mars: Season 1 (2004)
139660	4.5934	The Shawshank Redemption: Special Edition (1994)
89	4.5955	Tenchi Muyo! Ryo Ohki (1995)
25	4.6000	Trailer Park Boys: Season 4 (2003)
75	4.6000	Trailer Park Boys: Season 3 (2003)
1633	4.6050	Fullmetal Alchemist (2004)
1747	4.6388	Battlestar Galactica: Season 1 (2004)
7249	4.6710	Lost: Season 1 (2004)
74912	4.7026	Lord of the Rings: The Two Towers: Extended Edition (2002)
73422	4.7166	The Lord of the Rings: The Fellowship of the Ring: Ext

In [417]:
# analyze the one guy who has seen 17,563 movies (WTF)
hasnt_watched = []
last = 0
for i in sparse.find(non_zero_users_csr[55373])[1]:
    if i - last > 1:
        for j in range(1, i - last)[::-1]:
            hasnt_watched.append(i - j)
    last = i

# sort by average star rating
hasnt_watched = sorted(hasnt_watched, key=lambda x: review_nums[x][2], reverse=True)

In [418]:
# transpose back to (movie, user) orientation for effcient operations later
ratings_small = non_zero_users_csc
ratings_small = ratings_small.transpose()

# Add in rows for each genre
arr = np.array(movie_genres)
arr = np.insert(arr, 0, 0, axis=0)  # add zero row to match the zero row in ratings_small
csr_arr = csr_matrix(arr[:,1:])     # chop off the movie IDs and make it a csr matrix
ratings_small_with_genres = sparse.hstack([ratings_small, csr_arr], format = 'csr')  # append

In [ ]:
# MAIN REGRESSION CELL

max_reviews = 500                        # how many features are allowed in the regression
movie_results_dict = {}                  # destination dict for metrics/results
# top_reviewers, random_sample
choosing_mechanism = 'random_sample'     # choose how to pick the features
data = ratings_small_with_genres         # choose the matrix to regress on

# pointers to the indexes of the genre features (not review data) appended at the end
end_genre_index = ratings_small_with_genres.shape[1]
num_genres = 27
beg_genre_index = end_genre_index - num_genres
genre_indexes = range(beg_genre_index, end_genre_index)

# Loop over movie IDs, generating a model for each movie
for movie_id in range(10000,10500):
    # keep track of what movie you're on
    num_reviews = data[movie_id,:beg_genre_index].count_nonzero()
    movie_name = movie_by_id[movie_id]
    print 'Movie #%s, %s\naverage rating: %.2f in %i reviews  | ' % (
        movie_id,
        movie_name[:40],
        np.sum(data[movie_id,:beg_genre_index]) / num_reviews,
        int(num_reviews)
    ),
    
    ### PART I.
    ### MAKE THE X, y TO FEED THE REGRESSION OUT OF THE REVIEW DATA
    start = time.time()
    
    # filter out the reviewers who havent seen the movie
    user_mask = sparse.find(data[movie_id,:beg_genre_index])[1]
    
    # if there are many reviewers, only take some (based on choosing_mechanism)
    if (num_reviews > max_reviews):
        if (choosing_mechanism == 'top_reviewers'):
            user_mask = sorted(user_mask, key=lambda u: reviews_by_user[u])[:max_reviews]
        elif (choosing_mechanism == 'random_sample'):
            user_mask = user_mask[np.random.choice(len(user_mask), size=max_reviews)]
            
    # Include the genre features
    #user_mask = np.unique(np.hstack([genre_indexes, user_mask]))
    
    print 'regressing on %i features' % len(user_mask)
    
    # apply mask to filter the users (axis 0) of the X matrix
    ratings_filtered_by_user = data[:,user_mask]
    
    # make mask to filter out only the reviews for the movie in question
    movie_mask = np.ravel(np.full((ratings_filtered_by_user.shape[0], 1), True))
    movie_mask[movie_id] = False
    
    # generate X and y, training and testing data splitting
    X = ratings_filtered_by_user[movie_mask]
    y = ratings_filtered_by_user[movie_id]
    X_train, X_test, y_train, y_test = train_test_split(X.transpose().todense(), y.transpose().todense(), test_size=0.2, random_state=10)

    finish = time.time()
    data_time = (finish - start)

    ### PART II
    ### RUN THE REGRESSION ON THE FILM TO MAKE THE PREDICTOR
    start = time.time()
    
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)

    current_mse = mean_squared_error(y_test, y_pred)
    current_r2 = r2_score(y_test, y_pred)
    
    
    finish = time.time()
    regr_time = (finish - start)
    
    # add data to results dictionary
    movie_results_dict[id] = {
        'name': movie_by_id[movie_id],
        'regr_time': regr_time,
        'data_time': data_time,
        'regr': regr,
        'mse': mean_squared_error(y_test, y_pred),
        'r2': r2_score(y_test, y_pred),
    }
    print "MSE: %.2f \t\t r2: %.2f \t\t time: %.2f ... (%.2f + %.2f)" % (
        current_mse,
        current_r2,
        data_time + regr_time, data_time, regr_time
    )
    print

Movie #10000, Dr. Jekyll and Ms. Hyde (1995)
average rating: 2.90 in 215 reviews  |  regressing on 215 features
MSE: 1.11 		 r2: 0.21 		 time: 5.84 ... (0.57 + 5.27)

Movie #10001, Chef!: Series 2 (1994)
average rating: 3.80 in 158 reviews  |  regressing on 158 features
MSE: 1.31 		 r2: 0.07 		 time: 3.22 ... (0.52 + 2.69)

Movie #10002, The Three Stooges: Dizzy Doctors (1937)
average rating: 3.44 in 2624 reviews  |  regressing on 500 features
MSE: 1.50 		 r2: 0.24 		 time: 2.16 ... (0.59 + 1.57)

Movie #10003, Charlie Chan and the Curse of the Dragon
average rating: 2.47 in 107 reviews  |  regressing on 107 features
MSE: 1.36 		 r2: 0.06 		 time: 1.87 ... (0.51 + 1.35)

Movie #10004, Ninja Scroll (1995)
average rating: 4.21 in 4021 reviews  |  regressing on 500 features
MSE: 0.75 		 r2: -0.11 		 time: 2.41 ... (0.77 + 1.64)

Movie #10005, Typhoid Mary: The Most Dangerous Woman i
average rating: 3.25 in 269 reviews  |  regressing on 269 features
MSE: 1.02 		 r2: -0.15 		 time: 9.47 ...

In [235]:
# cursory analytics
def average_nested_dict_key(d, k):
    sum = np.sum([d[i][k] for i in d.keys()])
    return sum / len(d.keys())
        

ks = movie_results_dict.keys()
# sort by r2 value
print 'r2'
s = sorted(ks, key=lambda x: movie_results_dict[x]['r2'], reverse=True)
for i in s[:10]:
    print '%.3f\t%s' % (movie_results_dict[i]['r2'], movie_results_dict[i]['name'])

print
print 'average r2 value: %.2f' % average_nested_dict_key(movie_results_dict, 'r2')
    
# sort by mse
print '\nMSE'
s = sorted(ks, key=lambda x: movie_results_dict[x]['mse'])
for i in s[:10]:
    print '%.3f\t%s' % (movie_results_dict[i]['mse'], movie_results_dict[i]['name'])
    
print
print 'average MSE value: %.2f' % average_nested_dict_key(movie_results_dict, 'mse')

r2
0.254	Cadfael: The Potter's Field (1998)
0.249	Royal Tramp (1992)
0.247	The Three Stooges: Merry Mavericks (1951)
0.238	WWE: Judgment Day 2004 (2004)
0.223	In Praise of Love (2001)
0.221	Wanted: Dead or Alive: Season 1 (1958)
0.219	Dragon Ball Z: World Tournament (2001)
0.104	The Inheritance (2003)
0.073	Warren Miller's Journey (1990)
0.071	The Boys Next Door (1986)

average r2 value: -0.26

MSE
0.503	Ghost in the Shell: Stand Alone Complex: 2nd Gig (2005)
0.664	Cadfael: The Potter's Field (1998)
0.675	Little Shots of Happiness (2000)
0.807	Wanted: Dead or Alive: Season 1 (1958)
0.920	The Inheritance (2003)
0.993	Last of the Mississippi Jukes (2003)
1.008	Pompatus of Love (1996)
1.013	Knightriders (1981)
1.038	A Million to Juan (1994)
1.047	National Geographic: Surviving Everest (1999)

average MSE value: 1.52


In [136]:
# scratch space for pickling data structures for safe keeping
import pickle
with open('movie_results_dict_1000_1500.pickle', 'wb') as f:
    pickle.dump(movie_results_dict, f)

In [366]:
# scratch space for comparing runs of regressions

print 'with genre'
print 'average r2 value: %.2f' % average_nested_dict_key(with_genre, 'r2')
print 'average mse value: %.2f' % average_nested_dict_key(with_genre, 'mse')

print 'without genre'
print 'average r2 value: %.2f' % average_nested_dict_key(without_genre, 'r2')
print 'average mse value: %.2f' % average_nested_dict_key(without_genre, 'mse')



with genre
average r2 value: -0.48
average mse value: 2.78
without genre
average r2 value: 0.03
average mse value: 1.37


In [365]:
without_genre = movie_results_dict

array([1, 2, 3, 4, 5, 6])